In [2]:
import pandas as pd
import ast
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col,  when, lower
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
import pyspark.sql.functions as F
from pyspark.sql.functions import lit
import json
import ast
from pyspark.sql import SparkSession
from pyspark.sql import Row
from datetime import datetime, timedelta
import sys
from configparser import ConfigParser
import time

In [3]:
from IPython.core.display import HTML,display
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
# Reading the config data
config=ConfigParser(interpolation=None)
config.read('config.ini')
#DB
DB = str(config['default']['INP_DB_NM_1'])
DB2 = str(config['default']['INP_DB_NM_2'])
#input tables
input_tbl1=str(config['default']['INP_TBL_NM_1'])
input_tbl2=str(config['default']['INP_TBL_NM_2'])
input_tbl3=str(config['default']['INP_TBL_NM_3'])
input_tbl4=str(config['default']['INP_TBL_NM_4'])

related_party_df=(spark.table(".".join([DB2,input_tbl2])))
cerlic_banks_df=(spark.table(".".join([DB2,input_tbl3])))
keywords_df=(spark.table(".".join([DB2,input_tbl4])))

/opt/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  app.launch_new_instance()


In [5]:
df=(spark.table(".".join([DB,input_tbl1]))).filter((F.col('data_dt')=='2023-11-21'))

In [23]:
#df.show()

+----------------+-------------------+-------+----+-----------+-----------+--------------------+-------------+--------------+------------------+-------------+--------------------+-------------------+----------------+--------+--------------------+----------+-----------+---------------+-----------+--------------------+----------------+-------------+--------------------+---------------+--------------------+--------------+---------------+---------------------+-----------------+---------+--------------------+------------+---------+----------+---------+-------+-------------------+---------+-------+----------+--------------+--------------------+-----------------+---------------+-----------------+----+----+----------+
|      txn_ref_no|           txn_date|txn_amt|mode|    channel|remitter_id|       remitter_name|remitter_type|remitter_class|remitter_sub_class|remitter_ifsc|  remitter_bank_name|remitter_account_no|remitter_cust_id|benef_id|          benef_name|benef_type|benef_class|benef_sub_c

In [7]:
# Select columns for dummy_transactions_df
selected_columns = ['remitter_name', 'benef_name', 'benef_bank_name', 'direction']  # Replace with actual column names
dummy_transactions_df = df.select(selected_columns)

In [6]:
# Load the provided CSV files

#keywords_df = pd.read_csv("classification_keywords.csv")
# Load additional CSV files for the second part of the logic
# cerlic_df = pd.read_excel('Modified_Cerlic_Banks.xlsx')
# cerlic_df1 = StructType([StructField(col_name, StringType(), True) for col_name in cerlic_df.columns])
# cerlic_banks_df=spark.createDataFrame(cerlic_df,schema=cerlic_df1)

# related_party_df = pd.read_excel('Related party statistics .xlsx')
# related_party_df1 = StructType([StructField(col_name, StringType(), True) for col_name in related_party_df.columns])
# related_party_df=spark.createDataFrame(related_party_df,schema=related_party_df1)

In [15]:
# Define clean string UDF
start_time = time.time()

def clean_string(s):
    return s.replace(" ", "").lower() if s else ""
clean_string_udf = udf(clean_string, StringType())

# Clean data
cerlic_banks_df = cerlic_banks_df.withColumn("Borrower_Name", clean_string_udf(cerlic_banks_df["Borrower_Name"]))
cerlic_banks_df = cerlic_banks_df.withColumn("Name_of_Lender_Bank", clean_string_udf(cerlic_banks_df["Name_of_Lender_Bank"]))
related_party_df = related_party_df.withColumn("Related_Party_Name", clean_string_udf(related_party_df["Related_Party_Name"]))
related_party_df = related_party_df.withColumn("Borrower_Name", clean_string_udf(related_party_df["Borrower_Name"]))

# Add cleaned versions of remitter_name and beneficiary_name to dummy_transactions_df
dummy_transactions_df = dummy_transactions_df.withColumn("Remitter Name", clean_string_udf(dummy_transactions_df["remitter_name"]))
dummy_transactions_df = dummy_transactions_df.withColumn("Beneficiary Name", clean_string_udf(dummy_transactions_df["benef_name"]))
dummy_transactions_df = dummy_transactions_df.withColumn("Beneficiary Bank Name", clean_string_udf(dummy_transactions_df["benef_bank_name"]))

# Function to classify the bank
def classify_bank_v2(beneficiary_bank_name, remitter_name):
    remitter_name_clean = clean_string(remitter_name)
    beneficiary_bank_name_clean = clean_string(beneficiary_bank_name)
    return (remitter_name_clean, beneficiary_bank_name_clean)

classify_bank_udf_new = udf(classify_bank_v2, StringType())

# Function to classify the linkage
def classify_linkage_v2(beneficiary_name, remitter_name):
    beneficiary_name_clean = clean_string(beneficiary_name)
    remitter_name_clean = clean_string(remitter_name)
    return (beneficiary_name_clean, remitter_name_clean)

classify_linkage_udf_new = udf(classify_linkage_v2, StringType())

# Apply classification
dummy_transactions_df = dummy_transactions_df.withColumn(
    "Bank Classification",
    when(col("direction") == "Out", classify_bank_udf_new("benef_bank_name", "remitter_name")).otherwise("Others")
)
dummy_transactions_df = dummy_transactions_df.withColumn(
    "Linkage Classification",
    when(col("direction") == "Out", classify_linkage_udf_new("benef_bank_name", "remitter_name")).otherwise("Others")
)


# Load the keywords from a JSON file
with open('corporate_keywords.json', 'r') as file:
    data = json.load(file)
    corporate_keywords = data['corporate_keywords']
# Step 2: Convert the pandas DataFrame to a dictionary for keyword lookup

keywords_rows = keywords_df.collect()

# Initialize the dictionary
keywords_dict = {}

# Iterate over the rows
for row in keywords_rows:
    keywords_str = str(row['KEYWORDS']).strip()
    try:
        if keywords_str.startswith("[") and keywords_str.endswith("]"):
            keywords_list = ast.literal_eval(keywords_str)
        else:
            keywords_list = [keyword.strip() for keyword in keywords_str.split(",")]
        for keyword in keywords_list:
            keywords_dict[keyword.lower()] = (row['CATEGORY_LEVEL1'], row['CATEGORY_LEVEL2'])
    except Exception as e:
        print("Skipping this row...\n")

# Function to set category levels
def set_category_levels(base_txn_text, benef_name):
    base_txn_text = '' if base_txn_text is None else base_txn_text.lower()
    benef_name = '' if benef_name is None else benef_name.lower()
    
    for keyword, (cat_level1, cat_level2) in keywords_dict.items():
        if keyword in base_txn_text or keyword in benef_name:
            return (cat_level1, cat_level2)
    
    return ('OTHER TRANSFER', 'OTHER')

# Correcting the returnType to be StructType
schema = StructType([
    StructField("category_level1", StringType(), False),
    StructField("category_level2", StringType(), False)
])

# Register UDF for category levels
category_udf = udf(set_category_levels, schema)

def classify_transaction(benef_ifsc, benef_account_no, source, benef_name):
    def contains_corporate_keyword(name):
        return any(keyword in name.lower() for keyword in corporate_keywords)
    
    is_corporate = contains_corporate_keyword(benef_name)
    
    # Check if benef_ifsc starts with "YESB"
    if benef_ifsc and benef_ifsc.startswith("YESB"):
        # If source is not None, then use it to determine the output
        if source == 'current':
            return 'ybl_corp'
        elif source == 'saving':
            return 'ybl_ind'
        # If source is None, then decide based on is_corporate
        else:
            return 'ybl_corp' if is_corporate else 'ybl_ind'
    # If benef_ifsc does not start with "YESB"
    else:
        return 'non_ybl_cor' if is_corporate else 'non_ybl_ind'

classify_transaction_udf = udf(classify_transaction, StringType())
# Add a check to see if 'source' column exists in DataFrame and if not, add it with default None values
if 'source' not in df.columns:
    df = df.withColumn('source', lit(None))

# Perform the classification with the modified UDF that handles None source
df = df.withColumn('cor_ind_benf', classify_transaction_udf(
    col('benef_ifsc'),
    col('benef_account_no'),
    col('source'),
    col('benef_name'))
)

# First, create the 'categories' struct column
df = df.withColumn('categories', category_udf(col('base_txn_text'), col('benef_name')))

# Now, you can access the struct fields category_level1 and category_level2 from 'categories'
df = df.withColumn('category_level1', col('categories')['category_level1'])

# Define the UDF to check if all words in remitter_name are in benef_name
def all_words_present(remitter_name, benef_name):
    remitter_words = set(remitter_name.lower().split(' '))  # Split by space explicitly
    benef_words = set(benef_name.lower().split(' '))        # Split by space explicitly
    return remitter_words.issubset(benef_words)

# Register the UDF with Spark
all_words_present_udf = udf(all_words_present, BooleanType())

# Add a new column that uses the UDF to determine if category_level2 should be 'Personal Transfer'
df = df.withColumn(
    'category_level2',
    when(
        all_words_present_udf(col('remitter_name'), col('benef_name')),
        'PERSONAL TRANSFER'
    ).otherwise(
        col('categories')['category_level2']
    )
)

df = df.drop('categories', 'source')

# Assuming df and dummy_transactions_df are already defined and have a column 'txn_ref_no'

# Perform the join
result_df = df.join(
    dummy_transactions_df,
    df.remitter_name == dummy_transactions_df.remitter_name,
    "left_outer"
).select(
    df["*"],
    dummy_transactions_df["Bank Classification"],
    dummy_transactions_df["Linkage Classification"]
)

# Show the result
result_df.show(truncate=False)
end_time = time.time()
# Calculate the difference in seconds and then convert to minutes
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60.0

print("Execution time in minutes: {:.2f}".format(execution_time_minutes))

Skipping this row...

+----------------+-------------------+-------+----+-----------+-----------+------------------------------+-------------+--------------+------------------+-------------+------------------+-------------------+----------------+--------+--------------------------------+----------+-----------+---------------+-----------+-------------------+----------------+-------------+-----------------------------------------------------+---------------+-----------------------------+--------------+---------------+---------------------+-----------------+---------+--------------------------------+------------+---------+----------+---------+-------+-------------------+---------+-------+----------+--------------+--------------------+-----------------+---------------+-----------------+----+----+----------+------------+---------------+-----------------+-------------------+----------------------+
|txn_ref_no      |txn_date           |txn_amt|mode|channel    |remitter_id|remitter_name       

#### level3

In [8]:
# Define clean string UDF
start_time = time.time()

def clean_string(s):
    return s.replace(" ", "").lower() if s else ""
clean_string_udf = udf(clean_string, StringType())

# Clean data
cerlic_banks_df = cerlic_banks_df.withColumn("Borrower_Name", clean_string_udf(cerlic_banks_df["Borrower_Name"]))
cerlic_banks_df = cerlic_banks_df.withColumn("Name_of_Lender_Bank", clean_string_udf(cerlic_banks_df["Name_of_Lender_Bank"]))
related_party_df = related_party_df.withColumn("Related_Party_Name", clean_string_udf(related_party_df["Related_Party_Name"]))
related_party_df = related_party_df.withColumn("Borrower_Name", clean_string_udf(related_party_df["Borrower_Name"]))

# Add cleaned versions of remitter_name and beneficiary_name to dummy_transactions_df
dummy_transactions_df = dummy_transactions_df.withColumn("Remitter Name", clean_string_udf(dummy_transactions_df["remitter_name"]))
dummy_transactions_df = dummy_transactions_df.withColumn("Beneficiary Name", clean_string_udf(dummy_transactions_df["benef_name"]))
dummy_transactions_df = dummy_transactions_df.withColumn("Beneficiary Bank Name", clean_string_udf(dummy_transactions_df["benef_bank_name"]))

# Function to classify the bank
def classify_bank_v2(beneficiary_bank_name, remitter_name):
    remitter_name_clean = clean_string(remitter_name)
    beneficiary_bank_name_clean = clean_string(beneficiary_bank_name)
    return (remitter_name_clean, beneficiary_bank_name_clean)

classify_bank_udf_new = udf(classify_bank_v2, StringType())

# Function to classify the linkage
def classify_linkage_v2(beneficiary_name, remitter_name):
    beneficiary_name_clean = clean_string(beneficiary_name)
    remitter_name_clean = clean_string(remitter_name)
    return (beneficiary_name_clean, remitter_name_clean)

classify_linkage_udf_new = udf(classify_linkage_v2, StringType())

# Apply classification
dummy_transactions_df = dummy_transactions_df.withColumn(
    "Bank Classification",
    when(col("direction") == "Out", classify_bank_udf_new("benef_bank_name", "remitter_name")).otherwise("Others")
)
dummy_transactions_df = dummy_transactions_df.withColumn(
    "Linkage Classification",
    when(col("direction") == "Out", classify_linkage_udf_new("benef_bank_name", "remitter_name")).otherwise("Others")
)


# Load the keywords from a JSON file
with open('corporate_keywords.json', 'r') as file:
    data = json.load(file)
    corporate_keywords = data['corporate_keywords']
# Step 2: Convert the pandas DataFrame to a dictionary for keyword lookup

keywords_rows = keywords_df.collect()

# Initialize the dictionary
keywords_dict = {}

# Iterate over the rows
for row in keywords_rows:
    keywords_str = str(row['KEYWORDS']).strip()
    try:
        if keywords_str.startswith("[") and keywords_str.endswith("]"):
            keywords_list = ast.literal_eval(keywords_str)
        else:
            keywords_list = [keyword.strip() for keyword in keywords_str.split(",")]
        for keyword in keywords_list:
            keywords_dict[keyword.lower()] = (row['CATEGORY_LEVEL1'], row['CATEGORY_LEVEL2'], row['CATEGORY_LEVEL3'])
    except Exception as e:
        print("Skipping this row...\n")

# Function to set category levels including category_level3
def set_category_levels(base_txn_text, benef_name):
    base_txn_text = '' if base_txn_text is None else base_txn_text.lower()
    benef_name = '' if benef_name is None else benef_name.lower()
    
    for keyword, (cat_level1, cat_level2, cat_level3) in keywords_dict.items():
        if keyword in base_txn_text or keyword in benef_name:
            return (cat_level1, cat_level2, cat_level3)
    
    return ('OTHER TRANSFER', 'OTHER', 'N/A')

# Update the schema for category_udf to include category_level3
schema = StructType([
    StructField("category_level1", StringType(), False),
    StructField("category_level2", StringType(), False),
    StructField("category_level3", StringType(), False)  # New field for category_level3
])


# Register UDF for category levels
category_udf = udf(set_category_levels, schema)

def classify_transaction(benef_ifsc, benef_account_no, source, benef_name):
    def contains_corporate_keyword(name):
        return any(keyword in name.lower() for keyword in corporate_keywords)
    
    is_corporate = contains_corporate_keyword(benef_name)
    
    # Check if benef_ifsc starts with "YESB"
    if benef_ifsc and benef_ifsc.startswith("YESB"):
        # If source is not None, then use it to determine the output
        if source == 'current':
            return 'ybl_corp'
        elif source == 'saving':
            return 'ybl_ind'
        # If source is None, then decide based on is_corporate
        else:
            return 'ybl_corp' if is_corporate else 'ybl_ind'
    # If benef_ifsc does not start with "YESB"
    else:
        return 'non_ybl_cor' if is_corporate else 'non_ybl_ind'

classify_transaction_udf = udf(classify_transaction, StringType())
# Add a check to see if 'source' column exists in DataFrame and if not, add it with default None values
if 'source' not in df.columns:
    df = df.withColumn('source', lit(None))

# Perform the classification with the modified UDF that handles None source
df = df.withColumn('cor_ind_benf', classify_transaction_udf(
    col('benef_ifsc'),
    col('benef_account_no'),
    col('source'),
    col('benef_name'))
)

# First, create the 'categories' struct column
df = df.withColumn('categories', category_udf(col('base_txn_text'), col('benef_name')))\
       .withColumn('category_level1', col('categories')['category_level1'])\
       .withColumn('category_level2', col('categories')['category_level2'])\
       .withColumn('category_level3', col('categories')['category_level3'])  # New line to add category_level3

# Now, you can access the struct fields category_level1 and category_level2 from 'categories'
df = df.withColumn('category_level1', col('categories')['category_level1'])

# Define the UDF to check if all words in remitter_name are in benef_name
# Define the UDF to check if all words in remitter_name are in benef_name
def all_words_present(remitter_name, benef_name):
    if remitter_name is None or benef_name is None:
        return False
    remitter_words = set(remitter_name.lower().split(' '))  # Split by space explicitly
    benef_words = set(benef_name.lower().split(' '))        # Split by space explicitly
    return remitter_words.issubset(benef_words)

# Register the UDF with Spark
all_words_present_udf = udf(all_words_present, BooleanType())

# Add a new column that uses the UDF to determine if category_level2 should be 'Personal Transfer'
df = df.withColumn(
    'category_level2',
    when(
        all_words_present_udf(col('remitter_name'), col('benef_name')),
        'PERSONAL TRANSFER'
    ).otherwise(
        col('categories')['category_level2']
    )
)

df = df.drop('categories', 'source')

# Assuming df and dummy_transactions_df are already defined and have a column 'txn_ref_no'

# Perform the join
result_df = df.join(
    dummy_transactions_df,
    df.remitter_name == dummy_transactions_df.remitter_name,
    "left_outer"
).select(
    df["*"],
    dummy_transactions_df["Bank Classification"],
    dummy_transactions_df["Linkage Classification"]
)

# Show the result
result_df.show(truncate=False)
end_time = time.time()
# Calculate the difference in seconds and then convert to minutes
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60.0

print("Execution time in minutes: {:.2f}".format(execution_time_minutes))

Skipping this row...

+----------------+-------------------+-------+----+-----------+-----------+------------------------------+-------------+--------------+------------------+-------------+------------------+-------------------+----------------+--------+--------------------------------+----------+-----------+---------------+-----------+-------------------+----------------+-------------+-----------------------------------------------------+---------------+-----------------------------+--------------+---------------+---------------------+-----------------+---------+--------------------------------+------------+---------+----------+---------+-------+-------------------+---------+-------+----------+--------------+--------------------+-----------------+---------------+-----------------+----+----+----------+------------+---------------+-----------------+---------------+-------------------+----------------------+
|txn_ref_no      |txn_date           |txn_amt|mode|channel    |remitter_id|remi